In [11]:
from torch import nn, optim
from torch.utils.data import DataLoader
import torch

# from sklearn.metrics import accuracy_score, classfication_report

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import numpy as np

from tqdm import tqdm

from IMDBDataset import IMDBDataset
from IMDBTokenizers import IMDBTokenizer
from model import IMDBLstm

# dataLoader
# tokenizer
# Model
# train
# showLossAndError


In [3]:
# 常量层
torch.manual_seed(1)


word_dim=100
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

In [4]:
imdbTokenizer = IMDBTokenizer(vocab_path='aclImdb/imdb.vocab', glove_path='../../glove.6B.100d.txt', word_dim=word_dim, special_tokens=['<PAD>', '<UNK>'])

400001it [00:22, 17727.74it/s]
100%|██████████| 62596/62596 [00:00<00:00, 192320.45it/s]


In [5]:
trainIMDBDataset = IMDBDataset('./aclImdb/train')
trainIMDBDataLoader = DataLoader(trainIMDBDataset, batch_size=16, shuffle=True)
trainSmallIMDBDataLoader = DataLoader(trainIMDBDataset, batch_size=1, shuffle=True)

testIMDBDataset = IMDBDataset('./aclImdb/test')
testIMDBDataLoader = DataLoader(testIMDBDataset, batch_size=8, shuffle=True)

共有pos数据 12500 条, neg数据 12500 条，共 25000 条
共有pos数据 12500 条, neg数据 12500 条，共 25000 条


In [12]:
imdbLSTM = IMDBLstm(imdbTokenizer, 100, 100, 50, True, num_labels=2, device=device)

In [13]:
imdbLSTM = imdbLSTM.to(device)

# 查看模型的各个层，设置初始化策略，设置参数
base_lr = 0.0003

embedding_parameters = list(map(id, imdbLSTM.embedding.parameters()))

base_params = filter(lambda p: id(p) not in embedding_parameters, imdbLSTM.parameters())

optimizer = optim.Adam([{'params': base_params}, 
                        {'params': imdbLSTM.embedding.parameters(), 'lr': base_lr * 0.1}], lr=base_lr,betas=(0.9,0.999))


criterion = nn.NLLLoss()

In [ ]:
epochs = 10
skip = 100
import copy
model = imdbLSTM

for epoch in range(epochs):
    
    pred_list = []
    label_list = []
    
    model = model.to(device)
    print("epoch {}/{}".format(epoch, epochs))
    
    epoch_total_loss = 0.0
    epoch_acc_num = 0
    epoch_total_num = 0
    
    best_model = copy.deepcopy(model.state_dict())
    best_model_acc = 0.0
    
    skip_loss = 0.
    skip_num = 0
    
    for i, (inputs, labels) in enumerate(trainIMDBDataLoader):
        labels = labels.to(device)
        
        inputs = [str(i) for i in inputs]
        imdbLSTM.train()
        outputs = imdbLSTM(inputs)
        loss = criterion(outputs, labels)
        
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        loss.backward()
        optimizer.step()
        
        optimizer.zero_grad()
        # _, preds = torch.max(outputs, 1)
        
        preds = torch.argmax(outputs, dim=-1)
        
        pred_list.append(preds.cpu().numpy())
        label_list.append(labels.cpu().numpy())
        
        # acc_num = torch.sum(preds==labels)
        
        epoch_total_loss += loss.item() * outputs.size(0)
        epoch_total_num += outputs.size(0)
        # epoch_acc_num += acc_num
        skip_loss += loss.item() * outputs.size(0)
        skip_num += outputs.size(0)
        
        
        if i % skip == 0:
            print_loss = skip_loss / skip_num
            print('Mini epoch: {}/{} loss {}'.format(i, len(trainIMDBDataLoader), print_loss))
            skip_num = 0
            skip_loss = 0.
    pred_list = np.concatenate(pred_list)
    label_list = np.concatenate(label_list)
    epoch_acc = np.sum(pred_list == label_list)
    print('Epoch: {}/{} loss:{}, acc {}'.format(epoch, epochs, epoch_total_loss/epoch_total_num, epoch_acc))
    
    if epoch_acc > best_model_acc:
        best_model = copy.deepcopy(model.state_dict())


epoch 0/10
Mini epoch: 0/1563 loss 0.6850757598876953
Mini epoch: 100/1563 loss 0.6948187148571014
Mini epoch: 200/1563 loss 0.6990398174524307
Mini epoch: 300/1563 loss 0.695785700082779
Mini epoch: 400/1563 loss 0.6946278411149979
Mini epoch: 500/1563 loss 0.694382107257843
Mini epoch: 600/1563 loss 0.6941672146320343
Mini epoch: 700/1563 loss 0.6966572093963623
Mini epoch: 800/1563 loss 0.6930254918336868
Mini epoch: 900/1563 loss 0.6937296295166016
Mini epoch: 1000/1563 loss 0.6960772353410721
Mini epoch: 1100/1563 loss 0.695562292933464
Mini epoch: 1200/1563 loss 0.6972582763433457
Mini epoch: 1300/1563 loss 0.6940839666128159
Mini epoch: 1400/1563 loss 0.6912143278121948
Mini epoch: 1500/1563 loss 0.6932866632938385
Epoch: 0/10 loss:0.6948609519195557, acc 12607
epoch 1/10
Mini epoch: 0/1563 loss 0.7048056721687317
Mini epoch: 100/1563 loss 0.6932789295911789
Mini epoch: 200/1563 loss 0.6934782177209854
Mini epoch: 300/1563 loss 0.693990421295166
Mini epoch: 400/1563 loss 0.69402